# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning Frameworks</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.8.1


## Lab - Batch Normalization com MXNet

Em Machine Learning em geral, é muito comum normalizarmos a camada de entrada ajustando e dimensionando os dados. 

Por exemplo, quando temos atributos (variáveis) com valores de 0 a 1 e alguns de 1 a 1000, devemos normalizá-los para acelerar o aprendizado (colocar todos os dados na mesma escala). Se a camada de entrada está se beneficiando da normalização, por que não fazer o mesmo com os valores nas camadas ocultas, que mudam o tempo todo? Ao fazer isso, poderíamos obter 10 vezes ou mais melhorias na velocidade de treinamento.

A normalização em lotes reduz a quantidade pela qual os valores das unidades ocultas mudam (mudança de covariância). Para explicar a mudança de covariância, considere como exemplo uma rede profunda para detecção de gatos. Treinamos nossos dados apenas nas imagens de gatos pretos. Portanto, se agora tentarmos aplicar essa rede a dados com gatos coloridos, não vamos nos sair bem. O conjunto de treinamento e o conjunto de previsão são imagens de gatos, mas diferem um pouco. Em outras palavras, se um algoritmo aprendeu algum mapeamento de X para Y e se a distribuição de X mudar, talvez seja necessário treinar novamente o modelo, tentando alinhar a distribuição de X com a distribuição de Y.

Além disso, a normalização em lote permite que cada camada de uma rede aprenda sozinha um pouco mais independentemente de outras camadas.

Ao usar Batch Normalization, podemos usar taxas de aprendizado mais altas porque a normalização em lote garante que não haja ativação muito alta ou muito baixa. 

Batch Normalization também reduz a adaptação excessiva (overfitting), porque apresenta alguns efeitos de regularização. Semelhante ao Dropout, essa técnica adiciona algum ruído às ativações de cada camada oculta. Portanto, se usarmos a normalização em lote, usaremos menos dropouts, o que é uma coisa boa, pois não perderemos muitas informações. No entanto, não devemos depender apenas da normalização de lotes para regularização; devemos usá-lo em conjunto com o Dropout.

É o que faremos agora neste Lab.

Referências:

<a href="https://arxiv.org/pdf/1502.03167v3.pdf">Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift</a>

<a href="http://www.deeplearningbook.com.br/">Deep Learning Book</a>

In [2]:
!nvidia-smi

Thu Dec  2 04:23:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.31       Driver Version: 462.31       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX150      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P8    N/A /  N/A |     68MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

You should consider upgrading via the 'd:\users\rapha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [8]:
# Instala o pacote MXNet com suporte a GPU
!pip install -q mxnet-cu102

ERROR: Could not find a version that satisfies the requirement mxnet-cu102 (from versions: none)
ERROR: No matching distribution found for mxnet-cu102
You should consider upgrading via the 'd:\users\rapha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [7]:
# Imports
import mxnet as mx
from mxnet import nd, autograd, gluon
from mxnet.gluon import nn, data
import matplotlib
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'mxnet'

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

In [ ]:
# Faremos o treinamento em GPU
ctx = mx.gpu()

### Carregando o Dataset

Criaremos um modelo de Deep Learning para classificação de imagens usando o dataset CIFAR10.

In [ ]:
# Função para transformar os dados
def transform(data, label):
    return nd.moveaxis(data.astype('float32'), 2, 0)/255.0, label.astype('float32')

In [ ]:
# Carrega os dados de treino
cifar_treino = data.vision.datasets.CIFAR10(train = True, transform = transform)

In [ ]:
# Carrega os dados de teste
cifar_teste = data.vision.datasets.CIFAR10(train = False, transform = transform)

In [ ]:
# Extrai uma imagem
image, label = cifar_treino[456]

In [ ]:
# Shape da imagem
image.shape

In [ ]:
# Shape do label
label.shape

### Definindo o DataLoader

O DataLoader extrai batches de dados para alimentar o modelo durante o treinamento.

In [ ]:
# Tamanho do batch
batch_size = 128

In [ ]:
# Carrega os batches de treino e teste
dados_treino = data.DataLoader(cifar_treino, batch_size, True)
dados_teste = data.DataLoader(cifar_teste, batch_size, False)

### Construindo o Modelo

Agora construímos o modelo de rede neural para classificação de imagens usando Batch Normalization.

Observe que o Batch Normalization deve vir antes do Dropout.

In [ ]:
# Cria o modelo
model = nn.Sequential()

# Primeira camada convolucional com Batch Normalization
model.add(nn.Conv2D(channels = 16, kernel_size = (5, 5)))
model.add(nn.BatchNorm())
model.add(nn.Activation('relu'))
model.add(nn.MaxPool2D(pool_size = 2, strides = 1))

# Segunda camada convolucional com Batch Normalization
model.add(nn.Conv2D(channels = 32, kernel_size = (5, 5)))
model.add(nn.BatchNorm())
model.add(nn.Activation('relu'))
model.add(nn.MaxPool2D(pool_size = 2, strides = 1))

# Terceira camada convolucional com Batch Normalization
model.add(nn.Conv2D(channels = 64, kernel_size = (5, 5)))
model.add(nn.BatchNorm())
model.add(nn.Activation('relu'))
model.add(nn.MaxPool2D(pool_size = 2, strides = 1))

# Flatten
model.add(nn.Flatten())

# Camada Densa com Batch Normalization
model.add(nn.Dense(512))
model.add(nn.BatchNorm())
model.add(nn.Activation('relu'))
model.add(nn.Dropout(0.5))

# Camada Densa com Batch Normalization
model.add(nn.Dense(256))
model.add(nn.BatchNorm())
model.add(nn.Activation('relu'))
model.add(nn.Dropout(0.5))

# Camada de saída
model.add(nn.Dense(10))

In [ ]:
# Inicializa os hiperpaâmetros
model.initialize(mx.init.Xavier(), ctx)

In [ ]:
# Modelo criado
model

Vamos verificar um sumário completo.

In [ ]:
# Sumário do modelo
model.summary(image.expand_dims(0).as_in_context(ctx))

Precisamos de uma função de custo e usaremos a SoftmaxCrossEntropyLoss.

In [ ]:
# Função de custo
objective = gluon.loss.SoftmaxCrossEntropyLoss()

E também precisamos de um otimizador para atualizar os pesos a cada passada.

In [ ]:
# Otimizador
optimizer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 0.001})

Por fim, usaremos a acurácia como métrica de avaliação do modelo.

In [ ]:
# Métrica do modelo
metric = mx.metric.Accuracy()

### Treinamento do Modelo

Agora treinamos o modelo.

In [ ]:
# Número de épocas e listas para erros e acurácias em cada época
epochs = 10
losses = []
accs = []

In [ ]:
# Loop de treinamento

print("\nIniciando o Treinamento...\n")

for epoch in range(epochs):
    
    # Inicializa o erro acumulado
    cumulative_loss = 0
    
    # Reset da métrica
    metric.reset()
    
    # Loop pelos batches de dados
    for batches, (features, labels) in enumerate(dados_treino, 1):
        
        # Envia dados de entrada e saída para a GPU
        features = features.as_in_context(ctx)
        labels = labels.as_in_context(ctx)
        
        # Executa a previsão do modelo e calcula o erro
        with autograd.record():
            output = model(features)
            loss = objective(output, labels)
            
        # Inicia o backpropagation
        loss.backward()
        
        # Atualiza os pesos para a próxima passada de treino
        optimizer.step(batch_size)
        
        # Acumula o erro médio
        cumulative_loss += loss.mean()
        
        # Calcula a métrica
        metric.update(labels, output)
        
    # Extrai a acurácia    
    acc = metric.get()[1]
    
    # Alimenta as listas de erro e acurácias
    losses.append(cumulative_loss.asscalar())
    accs.append(acc)
    
    # print
    print(f'Epoch: {epoch} | Erro: {cumulative_loss.asscalar()/(batches):.5f} | Acurácia: {acc:.5f}')

print("\nTreinamento Concluído.\n")

### Avaliando o Modelo

In [ ]:
# Zeramos a métrica
metric.reset()

In [ ]:
# Loop para previsões nos dados de teste
for features, labels in dados_teste:
    features = features.as_in_context(mx.gpu())
    labels = labels.as_in_context(mx.gpu())
    predictions = model(features)
    metric.update(labels, predictions)

In [ ]:
print(f'Acurácia em Teste: {metric.get()[1]:.5f}')

In [ ]:
# Plots
plt.plot(accs, c = 'g')
plt.title('Acurácia em Treino')
plt.show()

plt.plot(losses, c = 'r')
plt.title('Erro em Treino')
plt.show()

Batch Normalization podem ser uma boa opção quando precisarmos aumentar a precisão do nosso modelo, reduzindo o risco de overfitting.

# Fim